In [1]:
import pandas as pd
import sqlite3
import numpy as np
import os
import shutil

In [2]:
#read specific table and return dataframe
def read_db_table(table_n, connection):
    df = pd.read_sql_query("SELECT * FROM " + table_n, connection)
    return df

In [3]:
#run query
def run_query(query, outputdB):
    conn = sqlite3.connect(outputdB)
    c = conn.cursor()
    c.execute(query)
    conn.commit()
    conn.close()

In [4]:
def write_sql(df_table, sqlite_table, outputdB):
#code snippets from https://github.com/EnergyModels/temoatools/tree/master/temoatools

    df_table = pd.DataFrame(df_table)
    
    # Set-up sqlite connection
    conn = sqlite3.connect(outputdB)
    c = conn.cursor()

    #----------
    # sqlite file prep
    #----------

    # Create SQL command based on number of entries
    command = 'INSERT INTO ' + sqlite_table + ' VALUES (?'
    for i in range(len(df_table.columns)-1):
        command = command + ',?'
    command = command + ')'

    # Execute SQL command
    try:
        c.executemany(command,np.array(df_table))
    except:
        print(command)
        print(np.array(df_table))
        c.executemany(command, np.array(df_table))

    #----------
    # Save(commit) the changes, and close sqlite file
    #----------
    conn.commit()
    conn.close()

In [5]:
# Create New SQL File
outputdB = 'data_files/temoa_test_system.sqlite'
#outputdB = 'data_files/temoa_regional_test_system.sqlite'

outputdBold = 'data_files/temoa_test_system_single.sqlite'

In [6]:
#get all commands from SQL file
fd = open('data_files/temoa_test_system_single.sql', 'r')
sqlFile = fd.read()
fd.close()

# all SQL commands (split on ';')
sqlCommands = sqlFile.split(';')

In [7]:
#filter CREATE TABLE commands
iterval = 0
create_sqlCommands=[]
for command in sqlCommands:
    if 'CREATE TABLE' in command:
        command = command.replace('"region"','"regions"')
        create_sqlCommands.append(command)

In [8]:
#write to new SQL file
fd = open('data_files/temoa_test_system.sql', 'w')
sqlFile = fd.write(";\n".join(create_sqlCommands))
fd.close()

In [9]:
if os.path.isfile(outputdB):
    os.remove(outputdB)
os.system("sqlite3 " + outputdB + " < data_files/temoa_test_system.sql")

if os.path.isfile(outputdBold):
    os.remove(outputdBold)
os.system("sqlite3 " + outputdBold + " < data_files/temoa_test_system_single.sql")

0

In [10]:
dict_tables_cols = dict()
dict_tables_cols['Efficiency'] = 'efficiency'
dict_tables_cols['CapacityToActivity'] = 'c2a'
dict_tables_cols['CapacityFactorTech'] = 'cf_tech'
dict_tables_cols['CostInvest'] = 'cost_invest'
dict_tables_cols['CostFixed'] = 'cost_fixed'
dict_tables_cols['CostVariable'] = 'cost_variable'
dict_tables_cols['Demand'] = 'demand'
dict_tables_cols['DemandSpecificDistribution'] = 'dds'
dict_tables_cols['EmissionActivity'] = 'emis_act'
dict_tables_cols['ExistingCapacity'] = 'exist_cap'
dict_tables_cols['LifetimeLoanTech'] = 'loan'
dict_tables_cols['LifetimeTech'] = 'life'
dict_tables_cols['MinActivity'] = 'minact'
dict_tables_cols['StorageDuration'] = 'duration'
dict_tables_cols['TechInputSplit'] = 'ti_split'
dict_tables_cols['TechOutputSplit'] = 'to_split'
dict_tables_cols['technologies'] = 'tech_category'

dict_tables_cols['GlobalDiscountRate'] = 'rate'
dict_tables_cols['SegFrac'] = 'segfrac'
dict_tables_cols['EmissionLimit'] = 'emis_limit'
dict_tables_cols['commodities'] = 'commodities'
dict_tables_cols['commodity_labels'] = 'comm_label_desc'
dict_tables_cols['sector_labels'] = 'sector_labels'
dict_tables_cols['tech_curtailment'] = 'tech_curtailment'
dict_tables_cols['technology_labels'] = 'tech_labels'
dict_tables_cols['time_of_day'] = 't_day'
dict_tables_cols['time_period_labels'] = 't_period_labels'
dict_tables_cols['time_periods'] = 't_periods'
dict_tables_cols['time_season'] = 't_season'
dict_tables_cols['regions'] = 'regions'

for key, value in dict_tables_cols.items():
    col_changed = value
    table_name = key
    
    conn = sqlite3.connect(outputdBold)
    df = read_db_table(table_name, conn)
    conn.close()
    if value in ['rate', 'segfrac','commodities', 'comm_label_desc', 'sector_labels', 'tech_category',
                 'tech_curtailment', 'tech_labels', 't_day', 't_period_labels', 't_periods', 't_season']:
        df_o = df.copy()
    else:
        df1 = df.copy()
        df2 = df.copy()
        df2.loc[:,'regions'] = 'R2'
        if value in ['exist_cap', 'minact']:
            df1.loc[:,col_changed]*=0.7
            df2.loc[:,col_changed]*=0.3
        elif value in ['demand']:
            df1.loc[df1.demand_comm=='RH',col_changed]*=0.3
            df2.loc[df2.demand_comm=='RH',col_changed]*=0.7
            df1.loc[df1.demand_comm=='VMT',col_changed]*=0.7
            df2.loc[df2.demand_comm=='VMT',col_changed]*=0.3
        elif value in ['emis_limit']:
            df1.loc[:,col_changed]*=1
            df2.loc[:,col_changed]*=1.5
            df2.loc[:,'regions'] = 'global'
        elif value in ['efficiency']:
            df2.loc[:,col_changed]*=1
        elif value not in ['c2a', 'dds', 'loan', 'life', 'duration', 'flag', 'regions', 'emis_act']:
            df2.loc[:,col_changed]*=0.8
        if value not in ['regions']:
            decs = max(df2.loc[:,col_changed].map(str).str.split('.').apply(lambda x: len(x[1])))
            df2.loc[:,col_changed] = df2.loc[:,col_changed].round(decs)
        df_o = pd.concat([df1, df2])
        #if value=='emis_limit':
        #    df_o = df1
            #df_o.loc[:,'region'] = 'R2'
    write_sql(df_o,table_name, outputdB)

In [11]:
def add_rows_df(row, df):
    srs = pd.Series(row, index = df.columns)
    df = df.append(srs, ignore_index=True)
    return df

In [12]:
#add transmission data
for key, value in dict_tables_cols.items():
    col_changed = value
    table_name = key
    if value in ['exist_cap']:
        conn = sqlite3.connect(outputdB)
        df = read_db_table(table_name, conn)
        conn.close()
        ind = len(df)
        df = add_rows_df(['R1-R2', 'E_TRANS',2015,10,'GW',''],df)
        df = add_rows_df(['R2-R1', 'E_TRANS',2015,10,'GW',''],df)
        write_sql(df.iloc[ind:],table_name, outputdB)
    elif value in ['tech_category']:
        conn = sqlite3.connect(outputdB)
        df = read_db_table(table_name, conn)
        conn.close()
        ind = len(df)
        df = add_rows_df(['E_TRANS', 'p','electric','electric transmission',''],df)
        write_sql(df.iloc[ind:],table_name, outputdB)
    elif value in ['cost_variable']:
        conn = sqlite3.connect(outputdB)
        df = read_db_table(table_name, conn)
        conn.close()
        ind = len(df)
        df = add_rows_df(['R1-R2', '2020','E_TRANS', '2015','0.1','$M/PJ',''],df)
        df = add_rows_df(['R1-R2', '2025','E_TRANS', '2015','0.1','$M/PJ',''],df)
        df = add_rows_df(['R1-R2', '2030','E_TRANS', '2015','0.1','$M/PJ',''],df)
        df = add_rows_df(['R2-R1', '2020','E_TRANS', '2015','0.1','$M/PJ',''],df)
        df = add_rows_df(['R2-R1', '2025','E_TRANS', '2015','0.1','$M/PJ',''],df)
        df = add_rows_df(['R2-R1', '2030','E_TRANS', '2015','0.1','$M/PJ',''],df)
        write_sql(df.iloc[ind:],table_name, outputdB)
    if value in ['efficiency']:
        conn = sqlite3.connect(outputdB)
        df = read_db_table(table_name, conn)
        conn.close()
        ind = len(df)
        df = add_rows_df(['R1-R2','ELC', 'E_TRANS',2015,'ELC',0.9,''],df)
        df = add_rows_df(['R2-R1','ELC', 'E_TRANS',2015,'ELC',0.9,''],df)
        write_sql(df.iloc[ind:],table_name, outputdB)
    if value in ['life']:
        conn = sqlite3.connect(outputdB)
        df = read_db_table(table_name, conn)
        conn.close()
        ind = len(df)
        df = add_rows_df(['R1-R2', 'E_TRANS',30,''],df)
        df = add_rows_df(['R2-R1', 'E_TRANS',30,''],df)
        write_sql(df.iloc[ind:],table_name, outputdB)
    if value in ['c2a']:
        conn = sqlite3.connect(outputdB)
        df = read_db_table(table_name, conn)
        conn.close()
        ind = len(df)
        df = add_rows_df(['R1-R2', 'E_TRANS',31.54,''],df)
        df = add_rows_df(['R2-R1', 'E_TRANS',31.54,''],df)
        write_sql(df.iloc[ind:],table_name, outputdB)

#add trans tech to tech_exchange        
df = pd.DataFrame(columns = ['tech', 'tech_notes'])
df = add_rows_df(['E_TRANS', ''],df)
write_sql(df,'tech_exchange', outputdB)


In [13]:
conn = sqlite3.connect(outputdB)
with open(outputdB.replace('ite',''), 'w') as f:
    for line in conn.iterdump():
        if 'e+' in line or 'e-0' in line:
            val = [x for x in line.split(',') if 'e+' in x or 'e-0' in x][0]
            ls = line.split(',')
            ind = ls.index(val)
            ls[ind] = [str(round(float(x),2)) for x in line.split(',') if 'e+' in x or 'e-0' in x ][0]
            line = ','.join(ls)
        if 'CREATE TABLE' in line:
            f.write('\n' )
        f.write('%s\n' % line)
        
conn.close()